In [ ]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations, product
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
from sympy import Point, Polygon
from decorators import *
from smartprint import smartprint as sprint
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN
import mmcv
from mmtrack.apis import inference_mot, init_model as init_tracking_model
from torchreid.utils import FeatureExtractor as REIDFeatureExtractor
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))
    
SERVER_CACHE_DIR = '/mnt/ci-nas-cache/edulyzeV2/cache_compute_4/fixed_face'
os.makedirs(SERVER_CACHE_DIR,exist_ok=True)

track_analysis_meta_cache = f'{SERVER_CACHE_DIR}/analysis_tracking/meta_info'
base_dir = '/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face/'

track_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_tracking/session_tracking_info'
os.makedirs(track_analysis_session_data,exist_ok=True)

postprocessed_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/processed_id_maps'
os.makedirs(postprocessed_id_map_data_dir, exist_ok=True)

emb_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_emb/session_emb_info_new'
os.makedirs(emb_analysis_session_data,exist_ok=True)

embmatched_id_raw_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/embmatched_id_raw'
os.makedirs(embmatched_id_raw_data_dir,exist_ok=True)

embmatched_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/embmatched_id_maps'
os.makedirs(embmatched_id_map_data_dir, exist_ok=True)

cross_session_input_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/cross_session_input'
os.makedirs(cross_session_input_data_dir, exist_ok=True)

id_viz_cache_root = f'{SERVER_CACHE_DIR}/analysis_emb/session_matching_info_new'
os.makedirs(id_viz_cache_root, exist_ok=True)


## Get frame file data for all sessions

In [ ]:

frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
frame_file_data.keys()


In [ ]:
session_filter_list = [
 #    'classinsight-cmu_05681A_ghc_4301_201905011630',
 # 'classinsight-cmu_05681A_ghc_4301_201904171630',
 # 'classinsight-cmu_05681A_ghc_4301_201902201630',
 # 'classinsight-cmu_05681A_ghc_4301_201904101630',
 # 'classinsight-cmu_05681A_ghc_4301_201901231630',
                       
 # 'classinsight-cmu_05418A_ghc_4102_201902251200',
 # 'classinsight-cmu_05418A_ghc_4102_201904081200',
 # 'classinsight-cmu_05418A_ghc_4102_201905011200',
 # 'classinsight-cmu_05418A_ghc_4102_201904291200',
 # 'classinsight-cmu_05418A_ghc_4102_201904011200',
                       
 'classinsight-cmu_05748A_ghc_4101_201902141630',
 'classinsight-cmu_05748A_ghc_4101_201904021630',
 'classinsight-cmu_05748A_ghc_4101_201902051630',
 'classinsight-cmu_05748A_ghc_4101_201902281630',
 'classinsight-cmu_05748A_ghc_4101_201903071630',
                       
 # 'classinsight-cmu_21127J_ghc_4102_201904230930',
 # 'classinsight-cmu_21127J_ghc_4102_201903260930',
 # 'classinsight-cmu_21127J_ghc_4102_201904160930',
 # 'classinsight-cmu_21127J_ghc_4102_201904300930',
 # 'classinsight-cmu_21127J_ghc_4102_201903190930',
                       
 # 'classinsight-cmu_05410A_ghc_4301_201904151500',
 # 'classinsight-cmu_05410A_ghc_4301_201902251500',
 # 'classinsight-cmu_05410A_ghc_4301_201904081500',
 # 'classinsight-cmu_05410A_ghc_4301_201904221500',
 # 'classinsight-cmu_05410A_ghc_4301_201902181500',
                       
 'classinsight-cmu_17214B_ph_a21_201902271030',
 'classinsight-cmu_17214B_ph_a21_201903061030',
 'classinsight-cmu_17214B_ph_a21_201904031030',
 'classinsight-cmu_17214B_ph_a21_201904101030',
 'classinsight-cmu_17214B_ph_a21_201904241030',
                       
 'classinsight-cmu_17214C_ph_225b_201903201130',
 'classinsight-cmu_17214C_ph_225b_201904101130',
 'classinsight-cmu_17214C_ph_225b_201904171130',
 'classinsight-cmu_17214C_ph_225b_201904241130',
 'classinsight-cmu_17214C_ph_225b_201905011130',
                       
 'classinsight-cmu_05410B_ghc_4211_201902111500',
 'classinsight-cmu_05410B_ghc_4211_201903181500',
 'classinsight-cmu_05410B_ghc_4211_201904081500',
 'classinsight-cmu_05410B_ghc_4211_201904151500',
 'classinsight-cmu_05410B_ghc_4211_201904221500',
 'classinsight-cmu_05410B_ghc_4211_201901281500'
]


# Get data for sessionA

In [ ]:
sample_course = '17214B'
sample_session_id = 'classinsight-cmu_17214B_ph_a21_201902271030-front'
session_tracking_cache_file = f"{track_analysis_session_data}/{sample_session_id}.pb"
session_preprocessed_id_map_file = f"{postprocessed_id_map_data_dir}/{sample_session_id}.pb"
session_frame_dir = f'{base_dir}/{sample_course}/{sample_session_id}'
session_video_file = f'/mnt/ci-nas-classes/classinsight/2019S/video_backup/{sample_session_id.split("-front")[0]}/{sample_session_id}.avi'
session_frame_dir


In [ ]:
old_to_new_id_map = pickle.load(open(session_preprocessed_id_map_file,"rb"))
df_session_eligible_pairs = pd.read_csv(f"{embmatched_id_map_data_dir}/{sample_session_id}.csv")
# Correct new ids with dict from eligible pairs
eligible_id_map_dict = {}
for id_pair in df_session_eligible_pairs.id_pair.values:
    (id1, id2) = eval(id_pair)
    print(id1, id2)
    if id2 in eligible_id_map_dict:
        eligible_id_map_dict[id1] = eligible_id_map_dict[id2]
    else:
        eligible_id_map_dict[id2] = id1

sprint(eligible_id_map_dict)

# correct created eligible map for once more
for key in sorted(list(eligible_id_map_dict.keys())):
    key_value = eligible_id_map_dict[key]
    if key_value in eligible_id_map_dict.keys():
        eligible_id_map_dict[key] = eligible_id_map_dict[key_value]
sprint(eligible_id_map_dict) 
# sprint({kr:old_to_new_id_map[kr] for kr in old_to_new_id_map if (not old_to_new_id_map[kr]==10000)}) 
sprint("replacing ids in old to new id maps")
for old_id in old_to_new_id_map:
    if old_to_new_id_map[old_id] in eligible_id_map_dict:
        print(f"replacing {old_id}:{old_to_new_id_map[old_id]} -->{eligible_id_map_dict[old_to_new_id_map[old_id]]}")
        old_to_new_id_map[old_id] = eligible_id_map_dict[old_to_new_id_map[old_id]]
# sprint({kr:old_to_new_id_map[kr] for kr in old_to_new_id_map if (not old_to_new_id_map[kr]==10000)}) 


In [ ]:
# map processed ids to tracking df
df_tracking_new = pickle.load(open(session_tracking_cache_file,"rb")).transpose()
total_idxs = df_tracking_new.index.max()
for old_id in old_to_new_id_map:
    new_id = old_to_new_id_map[old_id]
    if not new_id==10000:
        new_id_col = f'N{new_id}'
        if new_id_col not in df_tracking_new:
            df_tracking_new[new_id_col] = None
        df_tracking_new[new_id_col] =  df_tracking_new[new_id_col].where(~df_tracking_new[new_id_col].isnull(), df_tracking_new[old_id])
    df_tracking_new = df_tracking_new.drop(old_id, axis=1)




In [ ]:
col_start_stop_idxs = []
for col in df_tracking_new.columns:
    one_idxs = df_tracking_new.index[np.where(df_tracking_new[col]==1)[0]].values
    col_start_stop_idxs.append([col, one_idxs.min(), one_idxs.max()])
df_id_start_stop = pd.DataFrame(col_start_stop_idxs, columns=['id','min_idx','max_idx'])
df_id_start_stop['total_idxs'] = df_id_start_stop['max_idx']-df_id_start_stop['min_idx']
df_id_start_stop['id'] = df_id_start_stop['id'].apply(lambda x: int(x[1:]))
df_id_start_stop


In [ ]:
_ = plt.figure(figsize=(20,15))
for row_idx, row in df_id_start_stop.iterrows():
    plt.axhline(y=row_idx, xmin=row['min_idx']/total_idxs,xmax=row['max_idx']/total_idxs)
plt.yticks(range(df_id_start_stop.shape[0]), range(df_id_start_stop.shape[0]))
plt.grid() 


In [ ]:
session_frames = df_tracking_new.index.values.tolist()
session_ids_covered = []
selected_frames = []
segments = []
segment_half_size = 50
for row_idx, row in df_id_start_stop.sort_values(by='total_idxs').iterrows():
    id_start, id_stop = row['min_idx'],row['max_idx']
    seg_mid = (id_start+id_stop)/2
    seg_start, seg_end = max(seg_mid-segment_half_size, 0), min(seg_mid+segment_half_size, total_idxs)
    is_id_covered = df_id_start_stop.apply(lambda row: (seg_end<=row['max_idx']) & (seg_start>=row['min_idx']),axis=1)
    seg_ids_covered = df_id_start_stop[is_id_covered]['id']
    new_ids_covered = [xr for xr in seg_ids_covered if xr not in session_ids_covered]
    if len(new_ids_covered)>0:        
        segment_frames = [xr for xr in session_frames if ((xr>=seg_start) and (xr<=seg_end))]
        selected_frames+=segment_frames
        segments.append((seg_start, seg_end, new_ids_covered, segment_frames))
        session_ids_covered+=new_ids_covered

sprint(segments)
    
    


In [ ]:
SOURCE_ROOT = '/home/prasoon/video_analysis/edusenseV2compute/compute/videoV3'
run_config = {
    'track_config':f'{SOURCE_ROOT}/configs/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half.py',
    'track_checkpoint':f'{SOURCE_ROOT}/models/mmlab/ocsort_yolox_x_crowdhuman_mot17-private-half_20220813_101618-fe150582.pth',
    'device':'cuda:1',
}


In [ ]:
# for given frame ranges get video frames and tracking results
all_frames = mmcv.VideoReader(session_video_file)

required_frame_ids = np.unique(sorted(selected_frames))
frame_data_dict = {}

for frame_idx, frame_img in enumerate(all_frames):
    if (frame_idx in required_frame_ids) & (frame_idx%3==0):
        frame_data_dict[frame_idx] = frame_img
    if frame_idx%10000==0:
        print(f"Looped {frame_idx} images")


In [ ]:
reid_extractor = REIDFeatureExtractor(model_name='osnet_x1_0',device='cuda')


In [ ]:
frames_with_reids = {}
for seg_idx, (seg_start, seg_end, new_ids_covered, segment_frames) in enumerate(segments):
    frame_num=0
    for frame_idx in segment_frames:
        frame_tracking_file = f"{session_frame_dir}/{frame_idx}.pb"
        if os.path.exists(frame_tracking_file) and (frame_idx in frame_data_dict):
            frame_num+=1
            frame_results = pickle.load(open(frame_tracking_file,"rb"))[1]
            for person_idx in range(len(frame_results)):
                old_id= int(frame_results[person_idx]['track_id'])
                frame_results[person_idx]['track_id'] =old_to_new_id_map[old_id]
            frame_img = deepcopy(frame_data_dict[frame_idx])

            if frame_results is not None:
                # face_results = deepcopy(frame_results)
                body_count = len(frame_results)
                body_frames = []
                body_indexes = []
                for body_index, tracking_info in enumerate(frame_results):
                    if type(tracking_info) == dict:
                        body_bbox = tracking_info['bbox']
                        X_TL, Y_TL, X_BR, Y_BR = body_bbox[:4].astype(int)
                        if ((Y_BR - Y_TL) < 5) | ((X_BR - X_TL) < 5):
                            print("Very small body space found, not running face detection...")
                            continue
                        if (X_TL<0) | (Y_TL<0) | (X_BR<0) | (Y_BR<0):
                            print(frame_idx, body_index, Y_TL,Y_BR, X_TL,X_BR, body_frame.shape)
                            print("Negative boundaries for bounding boxes, skipping...")
                            continue
    
                        body_frame = frame_img[Y_TL:Y_BR, X_TL:X_BR, :]
                        body_frames.append(body_frame)
                        body_indexes.append(body_index)
                        
                
                body_reid_features = reid_extractor(body_frames)
                for body_index, body_reid_embedding in zip(body_indexes, body_reid_features):
                    frame_results[body_index].update({
                        'reid_features': body_reid_embedding.detach().cpu().numpy()
                    })
                frames_with_reids[frame_idx] = deepcopy(frame_results)
                
    print(f"Done for segment {seg_idx}")
    # break
    
    

In [ ]:
frames_with_reids[44274][0]['reid_features'].shape


In [ ]:
session_results.keys(), sample_session_id


In [ ]:
session_results[sample_session_id]  = deepcopy(frames_with_reids)


In [ ]:
# extract information at track_id level
session_id_oriented_results = {}
for session_id in session_results:
    reid_info = {}
    for frame_number in session_results[session_id]:
        for person in session_results[session_id][frame_number]:
            trackId = person['track_id']
            if trackId not in reid_info:
                reid_info[trackId] = []
            # get  gaze info
            try:
                person_reid = person['reid_features']
                reid_info[trackId].append([frame_number]+list(person_reid))
            except:
                print("reid not found",frame_number, trackId)
                continue
    
    for id in reid_info:
        reid_info[id] =pd.DataFrame(reid_info[id], columns=['frame']+np.arange(512).tolist()).set_index('frame')
    session_id_oriented_results[session_id]= deepcopy(reid_info)


In [ ]:
session_id_oriented_results.keys()


In [ ]:
CLU_EPS = 15
CLU_MIN_PTS = 50
cluster_based_embA = {}
sessionA = 'classinsight-cmu_17214B_ph_a21_201904241030-front'
emb_info = session_id_oriented_results[sessionA]
for sid in session_id_oriented_results[sessionA]:
    emb_clu = DBSCAN(min_samples=CLU_MIN_PTS, eps=CLU_EPS)
    try:
        if emb_info[sid].shape[0]>25000:
            emb_clu.fit(emb_info[sid].sample(25000).values)    
        else:
            emb_clu.fit(emb_info[sid].values)
    except:
        emb_clu=None
    if (emb_clu is None) or (max(emb_clu.labels_)<0):
        sprint(f"All frames are outliers, not proceeding with id {sid}")
        continue
    best_cluster_id = pd.Series(emb_clu.labels_[emb_clu.labels_>=0]).value_counts().index[0]
    frames = emb_info[sid].iloc[emb_clu.labels_==best_cluster_id].index.values
    cluster_based_embA[sid] = np.median(emb_info[sid].loc[frames],axis=0)
    sprint(f"Got cluster embedding for {sid}")


In [ ]:
CLU_EPS = 15
CLU_MIN_PTS = 50
cluster_based_embB = {}
sessionB = 'classinsight-cmu_17214B_ph_a21_201902271030-front'
emb_info = session_id_oriented_results[sessionB]
for sid in session_id_oriented_results[sessionB]:
    emb_clu = DBSCAN(min_samples=CLU_MIN_PTS, eps=CLU_EPS)
    try:
        if emb_info[sid].shape[0]>25000:
            emb_clu.fit(emb_info[sid].sample(25000).values)    
        else:
            emb_clu.fit(emb_info[sid].values)
    except:
        emb_clu=None
    if (emb_clu is None) or (max(emb_clu.labels_)<0):
        sprint(f"All frames are outliers, not proceeding with id {sid}")
        continue
    best_cluster_id = pd.Series(emb_clu.labels_[emb_clu.labels_>=0]).value_counts().index[0]
    frames = emb_info[sid].iloc[emb_clu.labels_==best_cluster_id].index.values
    cluster_based_embB[sid] = np.median(emb_info[sid].loc[frames],axis=0)
    sprint(f"Got cluster embedding for {sid}")


In [ ]:
session_cluster_emb={
    sessionA:deepcopy(cluster_based_embA),
    sessionB:deepcopy(cluster_based_embB),
}


In [ ]:
# find matches

final_matches = []
for (sessionA, sessionB) in product(list(session_id_oriented_results.keys()), list(session_id_oriented_results.keys())):
    sessionA_key, sessionB_key = sessionA.split("_")[-1][4:8], sessionB.split("_")[-1][4:8]
    if not (sessionA==sessionB):
        # if sessionA not in session_matches:
        # match session A and session B based on gaze clustering
        match_scores_reid = {}
        for idA,idB in product(session_cluster_emb[sessionA].keys(), session_cluster_emb[sessionB].keys()):
            clu_embA, clu_embB = session_cluster_emb[sessionA][idA], session_cluster_emb[sessionB][idB]
            if (idA==10000) | (idB==10000):
                continue
            if idA not in match_scores_reid:
                match_scores_reid[idA] = {}
                
            if clu_embA is None or clu_embB is None:
                match_scores_reid[idA][idB] = np.inf
            else:
                match_distance = cdist(clu_embA.reshape(1,-1), clu_embB.reshape(1,-1))[0][0]
                match_scores_reid[idA][idB] = match_distance
        
        df_match_clu = pd.DataFrame(match_scores_reid) 
        for col in df_match_clu.columns:
            if col not in df_match_clu.columns:
                df_match_clu[col] = np.inf
            sessionB_matches = deepcopy(df_match_clu[col]).sort_values().head(3).index.values.tolist()
            for match_id in sessionB_matches:
                final_matches.append((sessionA_key, sessionB_key, f'{sessionA_key}_{col}', f'{sessionB_key}_{match_id}', match_scores_reid[col][match_id]))

df_final_matches = pd.DataFrame(final_matches, columns=['sessionA','sessionB','idA','idB','match_score_reid'])


In [ ]:
df_final_matches.head(30)


In [ ]:
df_final_matches.head(30)


In [ ]:
distance_matrix = cdist(emb_info[11].values,emb_info[11].values)


In [ ]:
distance_matrix.shape


In [ ]:
sns.heatmap(distance_matrix)
